## Setup

Requirements:

- Python 3.8
- CPLEX Studio 20.1.0
- docplex library configured - only works with Python 3.8

## Import dependencies

In [24]:
# namedtuple creates a tuple with a name
# https://docs.python.org/3/library/collections.html#collections.namedtuple
from collections import namedtuple

# Model: https://ibmdecisionoptimization.github.io/docplex-doc/mp/docplex.mp.model.html
# Model is a class to embed modeling objects
from docplex.mp.model import Model
# LinearExpr contains the negate() function to create negate a linear expression
from docplex.mp.linear import LinearExpr

# used for normalisation of vectors
import numpy as np

# get all possible combinations for staticLex
import itertools

## Initialise Data

In [2]:
# Initialise data
Unit = namedtuple('Unit', ['name', 'att_strength', 'recruit_time_in_seconds', 'food'])

axe_unit = Unit('axe', 45, 90, 1)
lc_unit = Unit('lc', 130, 360, 4)
ma_unit = Unit('ma', 150, 450, 5)
serk_unit = Unit('serk', 300, 1200, 6)
ram_unit = Unit('ram', 2, 480, 5)

units = [axe_unit, lc_unit, ma_unit, serk_unit, ram_unit]
barracks_units = [axe_unit, lc_unit, ma_unit, ram_unit]
hoo_units = [serk_unit]

## Start a new model

In [3]:
# Alternatively, I can write
# with Model() as mdl:
#   ... # do things here
m = Model(name = 'Max strength unit time')

### Decision variables

In [4]:
number_of_units = m.integer_var_dict(
    [u.name for u in units], name = 'number_of_units'
)
number_of_units

{'axe': docplex.mp.Var(type=I,name='number_of_units_axe'),
 'lc': docplex.mp.Var(type=I,name='number_of_units_lc'),
 'ma': docplex.mp.Var(type=I,name='number_of_units_ma'),
 'serk': docplex.mp.Var(type=I,name='number_of_units_serk'),
 'ram': docplex.mp.Var(type=I,name='number_of_units_ram')}

### Decision expressions

In [5]:
m.barracks_build_time = m.sum([number_of_units[u.name] * u.recruit_time_in_seconds for u in barracks_units])
m.hoo_build_time = m.sum([number_of_units[u.name] * u.recruit_time_in_seconds for u in hoo_units])

In [6]:
m.totalNegativeAttack = LinearExpr.negate(m.sum([number_of_units[u.name] * u.att_strength for u in units]))
m.totalNegativeAttack

docplex.mp.LinearExpr(-45number_of_units_axe-130number_of_units_lc-150number_of_units_ma-300number_of_units_serk-2number_of_units_ram)

In [7]:
m.totalFood = m.sum([number_of_units[u.name] * u.food for u in units])
m.totalFood

docplex.mp.LinearExpr(number_of_units_axe+4number_of_units_lc+5number_of_units_ma+6number_of_units_serk+5number_of_units_ram)

### Constraints

From an error I got when trying to add constraints: `Only <=, ==, >= are allowed`

In [8]:
ct_food_no_church = m.add_constraints([m.totalFood <= 20596, m.totalFood >= 20580])
ct_food_no_church

[docplex.mp.LinearConstraint[](number_of_units_axe+4number_of_units_lc+5number_of_units_ma+6number_of_units_serk+5number_of_units_ram,LE,20596),
 docplex.mp.LinearConstraint[](number_of_units_axe+4number_of_units_lc+5number_of_units_ma+6number_of_units_serk+5number_of_units_ram,GE,20580)]

In [9]:
ct_must_have_rams = m.add_constraint(number_of_units['ram'] >= 250)
ct_must_have_rams

docplex.mp.LinearConstraint[](number_of_units_ram,GE,250)

In [12]:
# Need to modify our 0 < totalBuildTime <= 4 * 7 * 24 * 3600 to 1 <= totalBuildTime <= 4 * 7 * 24 * 3600 due to docplex limitation
ct_build_time_less_than_4_weeks_barracks = m.add_constraints([1 <= m.barracks_build_time, (m.barracks_build_time <= 4 * 7 * 24 * 3600)])
ct_build_time_less_than_4_weeks_hoo = m.add_constraints([1 <= m.hoo_build_time, (m.hoo_build_time <= 4 * 7 * 24 * 3600)])
print(ct_build_time_less_than_4_weeks_barracks)
print(ct_build_time_less_than_4_weeks_hoo)

[docplex.mp.LinearConstraint[](90number_of_units_axe+360number_of_units_lc+450number_of_units_ma+480number_of_units_ram,GE,1), docplex.mp.LinearConstraint[](90number_of_units_axe+360number_of_units_lc+450number_of_units_ma+480number_of_units_ram,LE,2419200)]
[docplex.mp.LinearConstraint[](1200number_of_units_serk,GE,1), docplex.mp.LinearConstraint[](1200number_of_units_serk,LE,2419200)]


### Objectives

In [13]:
m.add_kpi(m.totalNegativeAttack, "Total negative attack strength")
m.add_kpi(m.barracks_build_time, "Barracks build time")
m.add_kpi(m.hoo_build_time, "Hall of Order build time")

DecisionKPI(name=Hall of Order build time,expr=1200number_of_units_serk)

### Solve and print results

It doesn't look like docplex.mp supports staticLexFull.

In [62]:
kpis_permutations = itertools.permutations([m.totalNegativeAttack, m.hoo_build_time, m.barracks_build_time], 3)
kpis_permutations = list(kpis_permutations)


# pretty names for results
kpis_permutations_names = []
for permutation in kpis_permutations:
    name = []
    for kpi in permutation:
        if kpi is m.totalNegativeAttack:
            name.append('totalNegativeAttack')
        if kpi is m.hoo_build_time:
            name.append('hoo_build_time')
        if kpi is m.barracks_build_time:
            name.append('barracks_build_time')
    kpis_permutations_names.append(name)
kpis_permutations_names

[['totalNegativeAttack', 'hoo_build_time', 'barracks_build_time'],
 ['totalNegativeAttack', 'barracks_build_time', 'hoo_build_time'],
 ['hoo_build_time', 'totalNegativeAttack', 'barracks_build_time'],
 ['hoo_build_time', 'barracks_build_time', 'totalNegativeAttack'],
 ['barracks_build_time', 'totalNegativeAttack', 'hoo_build_time'],
 ['barracks_build_time', 'hoo_build_time', 'totalNegativeAttack']]

In [66]:
for idx, kpis in enumerate(kpis_permutations):
    m.minimize_static_lex(kpis)
    m.solve()

    if m.get_solve_status() == None:
        print("Model is infeasible")
    else:
        print(kpis_permutations_names[idx])
        print(
            f"axe: {number_of_units['axe'].solution_value}"
            f", lc: {number_of_units['lc'].solution_value}"
            f", ma: {number_of_units['ma'].solution_value}"
            f", serk: {number_of_units['serk'].solution_value}"
            f", ram: {number_of_units['ram'].solution_value}"
            f", food: {sum([number_of_units[u.name].solution_value * u.food for u in units])}"
            f", time: {round(sum([number_of_units[u.name].solution_value * u.recruit_time_in_seconds for u in units]) / (24 * 3600), 2)} days"
        )
        print(' ')

['totalNegativeAttack', 'hoo_build_time', 'barracks_build_time']
axe: 7250.0, lc: 0, ma: 0, serk: 2016.0, ram: 250.0, food: 20596.0, time: 36.94 days
 
['totalNegativeAttack', 'barracks_build_time', 'hoo_build_time']
axe: 7250.0, lc: 0, ma: 0, serk: 2016.0, ram: 250.0, food: 20596.0, time: 36.94 days
 
['hoo_build_time', 'totalNegativeAttack', 'barracks_build_time']
axe: 19340.0, lc: 0, ma: 0, serk: 1.0, ram: 250.0, food: 20596.0, time: 21.55 days
 
['hoo_build_time', 'barracks_build_time', 'totalNegativeAttack']
axe: 19324.0, lc: 0, ma: 0, serk: 1.0, ram: 250.0, food: 20580.0, time: 21.53 days
 
['barracks_build_time', 'totalNegativeAttack', 'hoo_build_time']
axe: 7234.0, lc: 0, ma: 0, serk: 2016.0, ram: 250.0, food: 20580.0, time: 36.92 days
 
['barracks_build_time', 'hoo_build_time', 'totalNegativeAttack']
axe: 7234.0, lc: 0, ma: 0, serk: 2016.0, ram: 250.0, food: 20580.0, time: 36.92 days
 
